In [1]:
import src
from src.utils import create_gif, load_model
from torchvision.transforms import Compose, Lambda, ToPILImage
import numpy as np
import torch
%matplotlib inline

/Users/dhruvsrikanth/Work/Projects/MPCS/Quarter_4/Fundamentals of Deep Learning/Project/.venv/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
beta_start = 0.0001
beta_end = 0.02
timesteps = 200
image_size = 28
num_channels = 1
batch_size = 128
device = "cpu"
epoch = 2

In [3]:
scheduler = src.LinearScheduler(beta_start=beta_start, beta_end=beta_end, timesteps=timesteps)

In [4]:
reverse_transform = Compose([
     Lambda(lambda t: (t + 1) / 2),
     Lambda(lambda t: t.permute(1, 2, 0)), # CHW to HWC
     Lambda(lambda t: t * 255.),
     Lambda(lambda t: t.numpy().astype(np.uint8)),
     ToPILImage(),
])

forward_diffusion = src.ForwardDiffusion(sqrt_alphas_cumprod=scheduler.sqrt_alphas_cumprod, sqrt_one_minus_alphas_cumprod=scheduler.sqrt_one_minus_alphas_cumprod, reverse_transform=reverse_transform)


In [5]:
device = torch.device(device)

In [6]:
model = src.DDPM(n_features=image_size, in_channels=num_channels, channel_scale_factors=(1, 2, 4,))
load_model(epoch=epoch, model=model, device=device)

sampler = src.Sampler(betas=scheduler.betas, sqrt_one_minus_alphas_cumprod=scheduler.sqrt_one_minus_alphas_cumprod, sqrt_one_by_alphas=scheduler.sqrt_one_by_alphas, posterior_variance=scheduler.posterior_variance, timesteps=timesteps)


In [7]:
# Get samples
samples = sampler.sample(model=model, image_size=image_size, batch_size=batch_size, channels=num_channels)


In [ ]:
create_gif(samples=samples, image_size=image_size, num_channels=num_channels, timesteps=timesteps, n_images=4)